In [1]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re
import os
import sqlite3

In [2]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [3]:
def df_find_cases_without_duplication_in_path(directory_path):
    words_for_table = []  # Use a list to store repeated entries
    sujeito_list = [] # Initialize sujeito_list outside the loop
    curso_list = []  # Initialize curso_list outside the loop
    sexo_list = [] # Initialize sexo_list outside the loop
    ano_list = [] # Initialize ano_list outside the loop
    cidade_list = [] # Initialize cidade_list outside the loop


    # Iterate through all files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(directory_path, filename)
            text = read_docx(file_path)

            ### WORDS WITH PATTERN ###
            pattern = r'\S*\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S*'
            matches = re.findall(pattern, text)
            words_for_table.extend(matches)

            ###### SUJEITOS #######
            sujeito_match = re.search(r'\bSUJEITO:\s*(\S+)', text)
            if sujeito_match:
                sujeito_list.extend([sujeito_match.group(1)] * len(matches))
            else:
                sujeito_list.extend(['Not Specified'] * len(matches))
            
             ##### CURSO #########
            curso_match = re.search(r'\bCURSO:\s*(\S+)', text)
            if curso_match:
                curso_list.extend([curso_match.group(1)] * len(matches))
            else:
                curso_list.extend(['Not Specified'] * len(matches))

             ##### SEXO #######
            sexo_match = re.search(r'\bSEXO:\s*(\S+)', text)
            if sexo_match:
                sexo_list.extend([sexo_match.group(1)] * len(matches))
            else:
                sexo_list.extend(['Not Specified'] * len(matches))

            
             ####### ANO ########
            ano_match = re.search(r'\bANO:\s*(\S+)', text)
            if ano_match:
                ano_list.extend([ano_match.group(1)] * len(matches))
            else:
                ano_list.extend(['Not Specified'] * len(matches))

            ####### CIDADE ########
            cidade_match = re.search(r'\bCIDADE:\s*(\S+)', text)
            if cidade_match:
                cidade_list.extend([cidade_match.group(1)] * len(matches))
            else:
                cidade_list.extend(['Not Specified'] * len(matches))

    df = DataFrame({'palavra': words_for_table, 'sujeito': sujeito_list, 'curso': curso_list, 'sexo': sexo_list, 'ano': ano_list, 'cidade': cidade_list})

    # Remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)

    return df


In [32]:
def create_patalizacao(row, i):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
    
    patalizacao_cru = re.findall(pattern, row['palavra'])

    if row['patalização'] == '':
        if re.search(pattern_t, patalizacao_cru[i]):
            return '[t]'
        if re.search(pattern_tf, patalizacao_cru[i]):
            return '[tʃ]'
        if re.search(pattern_dz, patalizacao_cru[i]):
            return '[dʒ]'
        if re.search(pattern_de, patalizacao_cru[i]):
            return '[de]'
        if re.search(pattern_di, patalizacao_cru[i]):
            return '[dɪ]'

    return row['patalização']

In [45]:
def create_patalizacao_cru(row, i):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
    if row['patalizacao_cru'] == '':
        patalizacao_cru = re.findall(pattern, row['palavra'])

        return patalizacao_cru[i]
    return row['patalizacao_cru']

In [46]:
# Function to determine 'sonoridade' based on 'patalização' values
def determine_sonoridade(row):
    
    if row == '[t]' or row == '[tʃ]':
        return 'surda'
    elif row == '[de]' or row == '[dɪ]' or row == '[dʒ]':
        return 'sonora'
    
    return None


In [33]:
def create_vogais_alvo(row,i):
    vogais_alvo = []
    
    pattern_I = r'\[[^\]]*ɪ[^\]]*\]'
    pattern_e = r'\[[^\]]*e[^\]]*\]'


    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
    
    patalizacao_cru = re.findall(pattern, row['palavra'])

    if row['vogais_alvo'] == '':
        if re.search(pattern_I, patalizacao_cru[i]):
            return '[ɪ]'
        if re.search(pattern_e, patalizacao_cru[i]):
            return '[e]'
        
    return row['vogais_alvo']

In [8]:
def create_posicao_patalizacao(row):
    
    
    unico_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[[^\]]*\]$'

  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    row['copy_palavra'] = row['palavra']

    if re.search(unico_pattern, row['palavra']):
        row['posição_patalização'] = 'único'
        row['copy_palavra'] = re.sub(unico_pattern, '', row['copy_palavra'])

    elif re.search(comeco_pattern, row['palavra']):
        row['posição_patalização'] = 'comeco'
        row['copy_palavra'] = re.sub(comeco_pattern, '', row['copy_palavra'])

    elif re.search(meio_pattern_mais, row['palavra']):
        row['posição_patalização'] = 'meio'
        row['copy_palavra'] = re.sub(meio_pattern, '', row['copy_palavra'], count=1)
    
    elif re.search(final_pattern, row['palavra']):
        row['posição_patalização'] = 'final'
        row['copy_palavra'] = re.sub(final_pattern, '', row['copy_palavra'])
    
    else:
        row['posição_patalização'] = ''
    return row

In [16]:
def df_add_duplicated_matches(df):

    # Regular expression pattern
    pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

    
    # Find matches for each value in the column
    matches = df['palavra'].str.findall(pattern)
    
    # idenfitfy idexes where there is a match
    match_index = df[matches.apply(lambda x: len(x) > 1)].index
    
    #turn index object to list
    match_list = list(match_index)
    
    i = 0
    for element in match_list:

        element_used = element + i
        new_row_data = df.loc[element_used:element_used]
        new_row_data = new_row_data[['palavra','copy_palavra', 'sujeito', 'curso', 'sexo', 'ano', 'cidade']]

        df = concat([df.iloc[:element_used + 1], new_row_data, df.iloc[element_used + 1:]]).reset_index(drop=True)
        i = i + 1
    
    df = df.fillna('') 
    return df


In [49]:
def final_create_posicao_patalizacao(row):
  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ| patalizacao_cru,dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    

    if row['posição_patalização'] == '':


        if re.search(comeco_pattern, row['copy_palavra']) or re.search(meio_pattern_mais, row['copy_palavra']):
            row['posição_patalização'] = 'meio'

        elif re.search(final_pattern, row['copy_palavra']):
            row['posição_patalização'] = 'final'
    return row



In [50]:
def apply_cols(directory_path):
    df = df_find_cases_without_duplication_in_path(directory_path)

    df['patalização'] = ''
    df['patalização'] = df.apply(lambda row: create_patalizacao(row, 0), axis=1)
    df['patalizacao_cru'] = ''
    df['patalizacao_cru'] = df.apply(lambda row: create_patalizacao_cru(row, 0), axis=1)
    
    df['sonoridade'] = df['patalização'].apply(determine_sonoridade)

    df['vogais_alvo'] = ''
    df['vogais_alvo'] = df.apply(lambda row: create_vogais_alvo(row, 0), axis=1)
    df = df.apply(create_posicao_patalizacao, axis=1)
    df = df_add_duplicated_matches(df)

    df['patalização'] = df.apply(lambda row: create_patalizacao(row, 1), axis=1)
    df['patalizacao_cru'] = df.apply(lambda row: create_patalizacao_cru(row, 1), axis=1)
    df['sonoridade'] = df['patalização'].apply(determine_sonoridade)
    df['vogais_alvo'] = df.apply(lambda row: create_vogais_alvo(row, 1), axis=1)

    df = df.apply(final_create_posicao_patalizacao, axis=1)
    df = df.drop(columns=['copy_palavra'])

    
    return df


In [51]:
directory_path = "../data/"
df = apply_cols(directory_path)
df

,palavra,sujeito,curso,sexo,ano,cidade,patalização,patalizacao_cru,sonoridade,vogais_alvo,posição_patalização
0,[dʒɪ]fícil,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[dʒ],[dʒɪ],sonora,[ɪ],comeco
1,duran[tʃɪ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[tʃ],[tʃɪ],surda,[ɪ],final
2,[dʒɪ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[dʒ],[dʒɪ],sonora,[ɪ],comeco
3,[dʒɪ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[dʒ],[dʒɪ],sonora,[ɪ],comeco
4,[dʒɪ]fícil,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[dʒ],[dʒɪ],sonora,[ɪ],comeco
...,...,...,...,...,...,...,...,...,...,...,...
2190,es[tʃɪ]ve[ɹ(n)][dʒɪ]s,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[dʒ],[dʒɪ],sonora,[ɪ],meio
2191,[dʒɪ],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[dʒ],[dʒɪ],sonora,[ɪ],comeco
2192,[dʒɪ]zer,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[dʒ],[dʒɪ],sonora,[ɪ],comeco
2193,fo[ɹ(n)][tʃɪ][s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[tʃ],[tʃɪ],surda,[ɪ],meio


In [52]:
new_cross_table = pd.crosstab(df['patalizacao_cru'], df['posição_patalização'])
new_cross_table

posição_patalização,comeco,final,meio,único
patalizacao_cru,,,,
[de],4,7,5,17
[dɪ],6,6,1,5
[dʒɪ],661,195,125,0
[te],0,9,4,0
[tɪ],8,10,11,0
[tʃɪ],214,583,324,0


In [55]:
# Calculate row-wise percentages
new_df_percentages = new_cross_table.copy()
total = (new_cross_table['comeco'] + new_cross_table['final'] + new_cross_table['meio'] + new_cross_table['único'])
new_df_percentages['comeco (%)'] = (new_cross_table['comeco'] /total) * 100
new_df_percentages['final (%)'] = (new_cross_table['final'] / total) * 100
new_df_percentages['meio (%)'] = (new_cross_table['meio']/ total) * 100
new_df_percentages['único (%)'] = (new_cross_table['único']/ total) * 100

new_df_percentages

posição_patalização,comeco,final,meio,único,comeco (%),final (%),meio (%),único (%)
patalizacao_cru,,,,,,,,
[de],4,7,5,17,12.121212,21.212121,15.151515,51.515152
[dɪ],6,6,1,5,33.333333,33.333333,5.555556,27.777778
[dʒɪ],661,195,125,0,67.380224,19.877676,12.742100,0.000000
[te],0,9,4,0,0.000000,69.230769,30.769231,0.000000
[tɪ],8,10,11,0,27.586207,34.482759,37.931034,0.000000
[tʃɪ],214,583,324,0,19.090098,52.007136,28.902765,0.000000


In [37]:
df['patalizacao'] = df['patalização'].replace({'[de]': '[d]', '[dɪ]': '[d]'})
cross_table = pd.crosstab(df['patalizacao'], df['vogais_alvo'])
cross_table

vogais_alvo,[e],[ɪ]
patalizacao,,
[d],33,18
[dʒ],0,981
[t],13,29
[tʃ],0,1121


In [38]:
# Calculate row-wise percentages
df_percentages = cross_table.copy()
df_percentages['[e] (%)'] = (cross_table['[e]'] / (cross_table['[e]'] + cross_table['[ɪ]'])) * 100
df_percentages['[ɪ] (%)'] = (cross_table['[ɪ]'] / (cross_table['[e]'] + cross_table['[ɪ]'])) * 100

df_percentages

vogais_alvo,[e],[ɪ],[e] (%),[ɪ] (%)
patalizacao,,,,
[d],33,18,64.705882,35.294118
[dʒ],0,981,0.000000,100.000000
[t],13,29,30.952381,69.047619
[tʃ],0,1121,0.000000,100.000000


In [23]:
# Create a SQLite in-memory database
conn = sqlite3.connect(':memory:')

In [24]:

# Write the DataFrame to the SQLite database
df.to_sql('my_table', conn, index=False, if_exists='replace') 

2195

In [25]:
query = "SELECT * FROM my_table where patalização = ''"

In [26]:
result = pd.read_sql_query(query, conn)